# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Does final quality check on menu items.'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([fp])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')
```

In [3]:
def random_number_from_string(input_string) -> int:
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price of each dish '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Does final quality check on menu items.'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 1),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([dish_price])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')

Observation: No subtasks completed yet for the Assigned Task
Thoughts: Need to start by generating dish names and descriptions using the Creative Writer function
Subtask identified: Generate interesting dish names and descriptions for Italian food choices
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for Italian food choices'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks completed yet for the Assigned Task
Thoughts: To generate interesting dish names and descriptions for Italian food choices, we can use the LLM function to come up with creative ideas.
Subtask identified: Use the LLM function to generate interesting dish names and descriptions for Italian food choices
Getting LLM to perform the following task: Use the LLM function to generate interesting dish names and descriptions for Italian food choices
> As an Italian cuisine specialist, I have crafted a list of enticing dish names and de

In [9]:
output = my_agent.reply_user()

Based on the Italian food choices provided, I have generated a menu with 5 items including their names, descriptions, ingredients, and prices. Here is the menu:\\\n\\\n1. Spaghetti Carbonara\\\nDescription: A classic dish with creamy egg and pancetta sauce.\\\nIngredients: Spaghetti, eggs, pancetta, Parmesan cheese, black pepper, salt.\\\n\\\n2. Truffle Risotto\\\nDescription: Indulgent risotto made with aromatic truffle oil.\\\nIngredients: Arborio rice, chicken or vegetable broth, white wine, shallots, garlic, truffle oil, Parmesan cheese, butter, salt, pepper.\\\n\\\n3. Osso Buco\\\nDescription: Savory veal shank braised to perfection.\\\nIngredients: Veal shanks, carrots, celery, onion, garlic, tomato paste, white wine, chicken or beef broth, bay leaf, thyme, parsley, salt, pepper.\\\n\\\n4. Caprese Salad\\\nDescription: Refreshing salad with ripe tomatoes, fresh basil, and creamy mozzarella.\\\nIngredients: Ripe tomatoes, fresh basil, mozzarella cheese, extra virgin olive oil, bal

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Summary of progress for Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Based on the Italian food choices provided, I have generated a menu with 5 items including their names, descriptions, ingredients, and prices. Here is the menu:\\n\\n1. Spaghetti Carbonara\\nDescription: A classic dish with creamy egg and pancetta sauce.\\nIngredients: Spaghetti, eggs, pancetta, Parmesan cheese, black pepper, salt.\\n\\n2. Truffle Risotto\\nDescription: Indulgent risotto made with aromatic truffle oil.\\nIngredie

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [11]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [12]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')

In [13]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks completed yet for the assigned task
Thoughts: To generate Italian dish names and descriptions, we can use the LLM function to come up with creative ideas.
Subtask identified: Generate Italian dish names and descriptions using the LLM function.
Getting LLM to perform the following task: Generate Italian dish names and descriptions using the LLM function.
> As an Italian cuisine enthusiast, I have crafted a list of delectable dish names and descriptions for you to savor. Indulge in the rich flavors of "Ravioli alla Caprese" - delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction. Or perhaps you'd prefer the hearty "Osso Buco al Gremolata" - tender braised veal shanks simmered in a savory broth and topped with a zesty 

In [14]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Generate Italian dish names and descriptions using the LLM function.
{'Output': 'As an Italian cuisine enthusiast, I have crafted a list of delectable dish names and descriptions for you to savor. Indulge in the rich flavors of "Ravioli alla Caprese" - delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction. Or perhaps you\'d prefer the hearty "Osso Buco al Gremolata" - tender braised veal shanks simmered in a savory broth and topped with a zesty gremolata of lemon zest, garlic, and parsley. Treat your taste b

In [15]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Observation: The Italian dish names and descriptions have been generated using the LLM function.
Thoughts: The next step is to generate ingredients for the 5 Italian dishes. This can be achieved by using the "use_llm" function to generate the ingredients based on the dish names provided.
Subtask identified: Generate ingredients for the 5 Italian dishes.
Getting LLM to perform the following task: Generate ingredients for the 5 Italian dishes.
> As an Italian cuisine enthusiast, I have meticulously curated the ingredients for each of the 5 delectable dishes. Indulge in the rich flavors of "Ravioli alla Caprese" - delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction. For "Osso Buco al Gremolata", savor tender braised veal shanks simmered in a savory broth and topped wit

In [16]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Observation: The Italian dish names, descriptions, and ingredients have been generated using the LLM function.
Thoughts: Since the dish names and descriptions have been created, the next step is to generate prices for the 5 Italian dishes. This can be achieved by using the dish_price function to assign prices to each dish.
Subtask identified: Generate prices for the 5 Italian dishes.
Calling function dish_price with parameters {'list_of_dish_names': ['Ravioli alla Caprese', 'Osso Buco al Gremolata', 'Tiramisu', 'Pappardelle al Cinghiale', 'Saltimbocca alla Romana']}
> {'output_1': {'Ravioli alla Caprese': '$8', 'Osso Buco al Gremolata': '$3', 'Tiramisu': '$10', 'Pappardelle al Cinghiale': '$2', 'Saltimbocca alla Romana': '$3'}}

### Auto-summarising Subtasks Completed ###
I have successfully generated 5 menu items based on Italian food choices.

In [17]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Observation: The Italian dish names, descriptions, ingredients, and prices have been successfully generated for the menu items.
Thoughts: The final step is to choose only 3 Italian dishes to serve that will please Gordon Ramsay.
Subtask identified: Select the 3 Italian dishes that will please Gordon Ramsay from the 5 menu items generated.
Getting LLM to perform the following task: Select the 3 Italian dishes that will please Gordon Ramsay from the 5 menu items generated.
> Based on Gordon Ramsay\'s refined palate and high standards, the 3 Italian dishes that would please him from the 5 menu items generated are "Ravioli alla Caprese" with its delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, priced at $8; "Tiramisu" with its classic layers of espresso-soaked ladyfingers and velvety mas

In [18]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

 "Ravioli alla Caprese", "Delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction", "Pasta, mozzarella, tomatoes, basil, balsamic reduction", "$8" \n "Tiramisu", "Layers of espresso-soaked ladyfingers and velvety mascarpone cheese, dusted with cocoa for a decadent finish", "Espresso, ladyfingers, mascarpone cheese, cocoa", "$10" \n "Saltimbocca alla Romana", "Veal cutlets, prosciutto, sage leaves, and white wine", "Veal cutlets, prosciutto, sage leaves, white wine", "$3"


In [19]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Generate Italian dish names and descriptions using the LLM function.
{'Output': 'As an Italian cuisine enthusiast, I have crafted a list of delectable dish names and descriptions for you to savor. Indulge in the rich flavors of "Ravioli alla Caprese" - delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction. Or perhaps you\'d prefer the hearty "Osso Buco al Gremolata" - tender braised veal shanks simmered in a savory broth and topped with a zesty gremolata of lemon zest, garlic, and parsley. Treat your taste b

In [20]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

"Ravioli alla Caprese", "Delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction", "Pasta, mozzarella, tomatoes, basil, balsamic reduction", "$8" \n "Tiramisu", "Layers of espresso-soaked ladyfingers and velvety mascarpone cheese, dusted with cocoa for a decadent finish", "Espresso, ladyfingers, mascarpone cheese, cocoa", "$10" \n "Saltimbocca alla Romana", "Veal cutlets, prosciutto, sage leaves, and white wine", "Veal cutlets, prosciutto, sage leaves, white wine", "$3"


In [21]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format
"Ravioli alla Caprese", "Delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction", "Pasta, mozzarella, tomatoes, basil, balsamic reduction", "$8" \n "Tiramisu", "Layers of espresso-soaked ladyfingers and velvety mascarpone cheese, dusted with cocoa for a decadent finish", "Espresso, ladyfingers, mascarpone cheese, cocoa", "$10" \n "Saltimbocca alla Romana", "Veal cutlets, prosciutto, sage leaves, and white wine", "Veal cutlets, prosciutt

In [22]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ('Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format', '"Ravioli alla Caprese", "Delicate parcels of pasta filled with creamy mozzarella, fresh tomatoes, and fragrant basil, drizzled with a luscious balsamic reduction", "Pasta, mozzarella, tomatoes, basil, balsamic reduction", "$8" \\n "Tiramisu", "Layers of espresso-soaked ladyfingers and velvety mascarpone cheese, dusted with cocoa for a decadent finish", "Espresso, ladyfingers, mascarpone cheese, cocoa", "$10" \\n "Saltimbocca alla Romana", "Veal cutlets, prosciutto, sage leaves, and white wine", "Veal cutlets, prosciutto, sage leaves, white wine", "$3"')


In [23]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed: None
Is Task Completed: False
